In [2]:
import emoji
import pandas as pd
import numpy as np
import re
import string
import nltk
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.decomposition import PCA
stopwords_list = stopwords.words('english')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stopwords_list = [stemmer.stem(word) for word in stopwords_list]

#GRADIO
import gradio as gr

#BOOSTED DECISION TREE
import sklearn
from sklearn.datasets import make_classification

import pickle
from sklearn.ensemble import GradientBoostingClassifier

import joblib

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/isabellasri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
dataset = pd.read_csv(r'/Users/isabellasri/Desktop/emoji_data/DS/group1.csv')
dataset2 = pd.read_csv(r'/Users/isabellasri/Desktop/emoji_data/DS/group2.csv')
dataset3 = pd.read_csv(r'/Users/isabellasri/Desktop/emoji_data/DS/group3.csv')
dataset4 = pd.read_csv(r'//Users/isabellasri/Desktop/emoji_data/DS/group4.csv')
dataset5 = pd.read_csv(r'/Users/isabellasri/Desktop/emoji_data/DS/group5.csv')

dataset = pd.concat([dataset, dataset2, dataset3, dataset4, dataset5])

In [ ]:
#REMOVING SPECIAL CHARACTERS FROM TWEETS
special_characters = ['~', ':', "'", '+', '[', '\\', '@', '^',
                      '{', '%', '(', '-', '"', '*', '|', ',', '&', '<', '`', '}', '.', '_', '=', ']', '>', ';', '#', '$', ')','!','?', '/', '’', '“', '”', "…"]

myoldemolist = dataset.loc[:, "clean"].tolist()

def replace_special(myemolist, myspeciallist):
    for i in myspeciallist:
        for j in range(len(myemolist)):
            myemolist[j] = myemolist[j].replace(i, "")
    return myemolist

my_new_clean = replace_special(myoldemolist, special_characters)

dataset["myNewClean"] = my_new_clean

# before i dropped 'clean', but ill keep it in by deleting it from the line below

dataset = dataset.drop(columns = ['emo', 'handle', 'created_at'])

In [1]:
#removing exclamations and question marks from this list
#Accuracy: 0.7358334319176624

In [ ]:
#REORGANIZING AGE COLUMN
age_list = list(dataset['age'])
for i in range(len(age_list)):
    if age_list[i] == 3:
        age_list[i] = 2
    if age_list[i] == 5:
        age_list[i] = 3
    if age_list[i] == 4:
        age_list[i] = 3

dataset['age'] = age_list

In [ ]:
#REMOVING STOPWORDS FROM TWEETS
from nltk.corpus import stopwords
stop = stopwords.words('english')

dataset['myNewClean'] = dataset['myNewClean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
#tokenized, separated the emojis
from nltk.tokenize.casual import TweetTokenizer
t = TweetTokenizer()
dataset['tokenized'] = dataset.apply(lambda x: t.tokenize(x['myNewClean']), axis=1)

lang = pd.read_csv(r'/Users/isabellasri/Desktop/emoji_data/DS/lang.csv')
lang_list = lang['0'].to_list()
dataset['lang'] = lang_list

lang_list = ("en", "other")
dataset = dataset[dataset.lang.isin(lang_list)]

word_list = dataset['tokenized'].tolist()

def word_dict(word_list):
    word_dict = {}
    for tweet in word_list:
        for word in tweet:
            if word not in word_dict:
                word_dict[word] = 1
            else:
                word_dict[word] += 1
    return(word_dict)

word_list = word_dict(word_list)

word_df = pd.DataFrame(word_list.items())

In [ ]:
#TF IDF

tokenized_tweets = list(dataset['tokenized'])

def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False, max_features = 1000)  

tf_tweets = tfidf.fit_transform(tokenized_tweets)
tfidf_array = tf_tweets.toarray()

In [ ]:
# PCA
NUM_COMPONENTS = 1000
pca = PCA(NUM_COMPONENTS)
reduced = pca.fit_transform(tfidf_array)

variance_explained = np.cumsum(pca.explained_variance_)

fig, ax = plt.subplots(figsize=(15, 8))
plt.plot(range(NUM_COMPONENTS),variance_explained, color='r')
ax.grid(True)
plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance")
#It takes around 300 components to explain 60% of variance. While 300 components post that expl

In [ ]:
###GRADIENT BOOSTED DECISION TREE

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_array, dataset['age'], test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.ensemble import GradientBoostingClassifier

# Build the model
gbdt = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=6, random_state=42)

# Train the model on the training set
print('Begin gbdt' + '-' * 50)
gbdt.fit(X_train, y_train)
print('End gbdt' + '-' * 50)

from sklearn.metrics import accuracy_score

# Predict the classes of the test set
print('Begin prediction' + '-' * 50)
y_pred = gbdt.predict(X_test)
print('End prediction' + '-' * 50)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

In [ ]:
# Save the model

pickle.dump(gbdt, open('gbdt.pkl', 'wb'))

In [ ]:
pickled_gbdt = pickle.load(open('gbdt.pkl', 'rb'))

In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
def reshape_tfidf(tfidf_matrix, expected_size):
    # Get the current size of the TF-IDF matrix
    current_size = tfidf_matrix.shape[1]

    # If the current size is less than the expected size, add zero columns to the right of the matrix
    if current_size < expected_size:
        zero_columns = np.zeros((tfidf_matrix.shape[0], expected_size - current_size))
        padded_tfidf_matrix = np.hstack((tfidf_matrix, zero_columns))

    # If the current size is greater than or equal to the expected size, truncate the matrix to the expected size
    else:
        padded_tfidf_matrix = tfidf_matrix[:, :expected_size]

    return padded_tfidf_matrix

In [ ]:
#GRADIO - prediction 2

def predict(text):
    try:
        # Preprocess the input text
        text = text.lower()
        clean_text = replace_special([text], special_characters)[0]
        print(clean_text)
        tokens = word_tokenize(clean_text)
        filtered_text = [word for word in tokens if not word in stopwords.words()]
        
        # Compute the TF-IDF representation of the preprocessed text
        tfidf2 = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False, max_features=1000)  
        tf_str = tfidf2.fit_transform([filtered_text])
        tfidf2_array = tf_str.toarray()

        # Apply scaling and padding to the TF-IDF array
        preprocessed = scaler.fit_transform(tfidf2_array)
        padded_tfidf_text = reshape_tfidf(preprocessed, 1000)

        # Make a prediction using the GBDT model
        prediction = pickled_gbdt.predict(padded_tfidf_text)

        return prediction[0]

    except Exception as e:
        # Handle any exceptions raised by the function calls
        error_message = f"An error occurred during prediction: {str(e)}"
        return error_message
print(predict('thanks dad lol😭😭 .@mythical '))

In [ ]:
#GRADIO - interface

iface = gr.Interface(
    fn=predict,
    inputs=gr.inputs.Textbox(label="Input text"),
    outputs=gr.outputs.Label(label="Age Group"))

iface.launch()

In [ ]:
#F1 SCORE
from sklearn.metrics import precision_recall_fscore_support

# define the positive class
pos_label = 1

# calculate precision, recall, and F1 score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, pos_label=pos_label, average='weighted')

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1_score)